<a href="https://colab.research.google.com/github/leedhn/papago/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import pandas as pd #edit
import numpy as np
from glob import glob
import logging


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
from google.colab import drive #edit
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NAVER 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NAVER


#Logging

In [4]:
NUM_EPOCHS = 10
#OPTIMIZER = 'SGD'

logger = logging.getLogger('my_logger')

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger.info('This message has a date/time timestamp')


logger.propagate = False # do not pass logs to the default logger

# Create handlers
c_handler = logging.StreamHandler()
EPOCH=10
f_handler = logging.FileHandler(f'GRU_{NUM_EPOCHS}.log', mode='w')
c_handler.setLevel(logging.INFO)
f_handler.setLevel(logging.INFO)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(asctime)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)


2021-06-10 13:09:17,217 - This message has a date/time timestamp


#데이터 파일 로딩


In [5]:
SOS_token = 0
EOS_token = 1


# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {0: "SOS", 1: "EOS"}
#         self.n_words = 2  # SOS 와 EOS 포함

#     def addSentence(self, sentence):
#         for word in sentence.split(' '):
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             self.word2index[word] = self.n_words
#             self.word2count[word] = 1
#             self.index2word[self.n_words] = word
#             self.n_words += 1
#         else:
#             self.word2count[word] += 1

In [6]:
# 유니 코드 문자열을 일반 ASCII로 변환하십시오.
# https://stackoverflow.com/a/518232/2809427
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )

# # 소문자, 다듬기, 그리고 문자가 아닌 문자 제거


# def normalizeString(s):
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
#     return s

In [7]:
# def readLangs(lang1, lang2, reverse=False):
#     print("Reading lines...")

#     # 파일을 읽고 줄로 분리
#     lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
#         read().strip().split('\n')

#     # 모든 줄을 쌍으로 분리하고 정규화
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

#     # 쌍을 뒤집고, Lang 인스턴스 생성
#     if reverse:
#         pairs = [list(reversed(p)) for p in pairs]
#         input_lang = Lang(lang2)
#         output_lang = Lang(lang1)
#     else:
#         input_lang = Lang(lang1)
#         output_lang = Lang(lang2)

#     return input_lang, output_lang, pairs

In [8]:
MAX_LENGTH = 100

# eng_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s ",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

데이터 준비를 위한 전체 과정:

-  텍스트 파일을 읽고 줄로 분리하고, 줄을 쌍으로 분리합니다.
-  텍스트를 정규화 하고 길이와 내용으로 필터링 합니다.
-  쌍을 이룬 문장들로 단어 리스트를 생성합니다.




In [9]:
logger.info('='*50)
logger.info('LOADING DATA')
logger.info('')

%cd data
txt_list = glob('*.txt') #edit
datas = {}
for txt in txt_list:
    datas[txt] = pd.read_csv(txt,header=None)
    name = txt[-10:-4]
    datas[txt].columns = [name]
    for i in range(len(datas[txt][name])):
        datas[txt][name][i] = np.fromstring(datas[txt][name][i] ,dtype=int,sep=' ').tolist()
        datas[txt][name][i].append(1)
    logger.info(f'Load {txt} finished')
%cd ../
logger.info('='*50)

2021-06-10 13:09:17,262 - ==================================================
2021-06-10 13:09:17,264 - LOADING DATA
2021-06-10 13:09:17,267 - 


/content/drive/MyDrive/NAVER/data


2021-06-10 13:09:18,102 - Load train_source.txt finished
2021-06-10 13:09:18,916 - Load train_target.txt finished
2021-06-10 13:09:19,083 - Load test_target.txt finished
2021-06-10 13:09:19,232 - Load test_source.txt finished
2021-06-10 13:09:19,235 - ==================================================


/content/drive/My Drive/NAVER


In [10]:
# def prepareData(lang1, lang2, reverse=False):
#     input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
#     print("Read %s sentence pairs" % len(pairs))
#     pairs = filterPairs(pairs)
#     print("Trimmed to %s sentence pairs" % len(pairs))
#     print("Counting words...")
#     for pair in pairs:
#         input_lang.addSentence(pair[0])
#         output_lang.addSentence(pair[1])
#     print("Counted words:")
#     print(input_lang.name, input_lang.n_words)
#     print(output_lang.name, output_lang.n_words)
#     return input_lang, output_lang, pairs


# input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
# print(random.choice(pairs))

Seq2Seq 모델
=================

Recurrent Neural Network(RNN)는 시퀀스에서 작동하고 다음 단계의
입력으로 자신의 출력을 사용하는 네트워크입니다.

`Sequence to Sequence network <https://arxiv.org/abs/1409.3215>`__, 또는
Seq2Seq 네트워크, 또는 `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__ 는 인코더 및
디코더라고 하는 두 개의 RNN으로 구성된 모델입니다.
인코더는 입력 시퀀스를 읽고 단일 벡터를 출력하고,
디코더는 해당 벡터를 읽어 출력 시퀀스를 생성합니다.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

모든 입력에 해당하는 출력이 있는 단일 RNN의 시퀀스 예측과 달리
Seq2Seq 모델은 시퀀스 길이와 순서를 자유롭게하기 때문에
두 언어 사이의 번역에 이상적입니다.

다음 문장 "Je ne suis pas le chat noir" → "I am not the black cat"
를 살펴 봅시다. 입력 문장의 단어 대부분은 출력 문장에서
직역("chat noir" 와 "black cat")되지만 약간 다른 순서도 있습니다.
"ne/pas" 구조로 인해 입력 문장에 단어가 하나 더 있습니다.
입력 단어의 시퀀스를 직역해서 정확한 번역을 만드는
것은 어려울 것입니다.

Seq2Seq 모델을 사용하면 인코더는 하나의 벡터를 생성합니다.
이상적인 경우에 입력 시퀀스의 "의미"를 문장의 N 차원 공간에 있는
단일 지점인 단일 벡터으로 인코딩합니다.




인코더
-----------

Seq2Seq 네트워크의 인코더는 입력 문장의 모든 단어에 대해 어떤 값을
출력하는 RNN입니다. 모든 입력 단어에 대해 인코더는 벡터와
은닉 상태를 출력하고 다음 입력 단어를 위해 그 은닉 상태를 사용합니다.

.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:





In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

디코더
-----------

디코더는 인코더 출력 벡터를 받아서 번역을 생성하기 위한 단어 시퀀스를
출력합니다.




간단한 디코더
^^^^^^^^^^^^^^

가장 간단한 Seq2Seq 디코더는 인코더의 마지막 출력만을 이용합니다.
이 마지막 출력은 전체 시퀀스에서 문맥을 인코드하기 때문에
*문맥 벡터(context vector)* 로 불립니다. 이 문맥 벡터는 디코더의 초기 은닉 상태로
사용 됩니다.

디코딩의 매 단계에서 디코더에게 입력 토큰과 은닉 상태가 주어집니다.
초기 입력 토큰은 문자열-시작 (start-of-string) ``<SOS>`` 토큰이고,
첫 은닉 상태는 문맥 벡터(인코더의 마지막 은닉 상태) 입니다.

.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:





In [12]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

이 모델의 결과를 학습하고 관찰하는 것을 권장하지만,
공간을 절약하기 위해 최종 목적지로 바로 이동해서
Attention 메카니즘을 소개 할 것입니다.




Attention 디코더
^^^^^^^^^^^^^^^^^

문맥 벡터만 인코더와 디코더 사이로 전달 된다면, 단일 벡터가 전체 문장을
인코딩 해야하는 부담을 가지게 됩니다.

Attention은 디코더 네트워크가 자기 출력의 모든 단계에서 인코더 출력의
다른 부분에 "집중" 할 수 있게 합니다. 첫째 *Attention 가중치* 의 세트를
계산합니다. 이것은 가중치 조합을 만들기 위해서 인코더 출력 벡터와
곱해집니다. 그 결과(코드에서 ``attn_applied``)는 입력 시퀀스의
특정 부분에 관한 정보를 포함해야하고 따라서 디코더가 알맞은 출력
단어를 선택하는 것을 도와줍니다.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

어텐션 가중치 계산은 디코더의 입력 및 은닉 상태를 입력으로
사용하는 다른 feed-forwad 계층인 ``attn`` 으로 수행됩니다.
학습 데이터에는 모든 크기의 문장이 있기 때문에 이 계층을 실제로
만들고 학습시키려면 적용 할 수 있는 최대 문장 길이 (인코더 출력을 위한 입력 길이)를
선택해야 합니다. 최대 길이의 문장은 모든 Attention 가중치를 사용하지만
더 짧은 문장은 처음 몇 개만 사용합니다.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [13]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

학습
========

학습 데이터 준비
-----------------------

학습을 위해서, 각 쌍마다 입력 Tensor(입력 문장의 단어 주소)와
목표 Tensor(목표 문장의 단어 주소)가 필요합니다. 이 벡터들을
생성하는 동안 두 시퀀스에 EOS 토큰을 추가 합니다.




In [14]:
# def indexesFromSentence(lang, sentence):
#     return [lang.word2index[word] for word in sentence.split(' ')]


# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


# def tensorsFromPair(pair):
#     input_tensor = tensorFromSentence(input_lang, pair[0])
#     target_tensor = tensorFromSentence(output_lang, pair[1])
#     return (input_tensor, target_tensor)

모델 학습
------------------

학습을 위해서 인코더에 입력 문장을 넣고 모든 출력과 최신 은닉 상태를
추적합니다. 그런 다음 디코더에 첫 번째 입력으로 ``<SOS>`` 토큰과
인코더의 마지막 은닉 상태가 첫번쩨 은닉 상태로 제공됩니다.

"Teacher forcing"은 다음 입력으로 디코더의 예측을 사용하는 대신
실제 목표 출력을 다음 입력으로 사용하는 컨셉입니다.
"Teacher forcing"을 사용하면 수렴이 빨리되지만 `학습된 네트워크가
잘못 사용될 때 불안정성을 보입니다.
<http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf>`__.

Teacher-forced 네트워크의 출력이 일관된 문법으로 읽지만 정확한
번역과는 거리가 멀다는 것을 볼 수 있습니다. 직관적으로 출력 문법을
표현하는 법을 배우고 교사가 처음 몇 단어를 말하면 의미를 "선택" 할 수 있지만,
번역에서 처음으로 문장을 만드는 법은 잘 배우지 못합니다.

PyTorch의 autograd 가 제공하는 자유 덕분에 간단한 If 문으로
Teacher Forcing을 사용할지 아니면 사용하지 않을지를 선택할 수 있습니다.
더 많이 사용하려면 ``teacher_forcing_ratio`` 를 확인하십시오.




In [15]:
teacher_forcing_ratio = 0.5
MAX_LENGTH=100

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_hidden.to(device)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        
    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

이것은 현재 시간과 진행률%을 고려해 경과된 시간과 남은 예상
시간을 출력하는 헬퍼 함수입니다.




In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

전체 학습 과정은 다음과 같습니다:

-  타이머 시작
-  optimizers와 criterion 초기화
-  학습 쌍의 세트 생성
-  도식화를 위한 빈 손실 배열 시작

그런 다음 우리는 여러 번 ``train`` 을 호출하며 때로는 진행률
(예제의 %, 현재까지의 예상 시간)과 평균 손실을 출력합니다.




In [17]:
# source_words = []
# for line in datas['train_source.txt']['source']:
#     for num in line:
#         if num not in source_words:
#             source_words.append(num)

# len(source_words)

In [36]:
def trainIters(encoder, decoder, n_iters,logger, epochs, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화
    learning_rate = 0.001
    # encoder.train()
    # decoder.train()

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    # decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    # 옵티마이저의 state_dict 출력
    logger.info('Encoder Optimizer state_dict:')
    for var_name in encoder_optimizer.state_dict():
        logger.info(f'{var_name} \t {encoder_optimizer.state_dict()[var_name]}')
    logger.info('Decoder Optimizer state_dict:')
    for var_name in decoder_optimizer.state_dict():
        logger.info(f'{var_name} \t {decoder_optimizer.state_dict()[var_name]}')

    
    # training_pairs = [tensorsFromPair(random.choice(pairs))
    #                   for i in range(n_iters)]
    criterion = nn.NLLLoss()

    logger.info('='*50)
    logger.info('TRAIN')
    logger.info('-'*50)

    for epoch in range(epochs):
        encoder.train()
        decoder.train()
        
        logger.info(f'EPOCH : {epoch}')
        logger.info('-'*30)
        for iter in range(1, n_iters + 1):
        ##for iter in range(0, niters):
            # training_pair = training_pairs[iter - 1]
            # input_tensor = training_pair[0]
            # target_tensor = training_pair[1]
            input_list = datas['train_source.txt']['source'][iter-1]
            #input_list.append(EOS_token)

            target_list = datas['train_target.txt']['target'][iter-1]
            #target_list.append(EOS_token)
            input_tensor = torch.Tensor(input_list).type(torch.long).unsqueeze(1)
            input_tensor =input_tensor.to(device)
            target_tensor = torch.Tensor(target_list).type(torch.long).unsqueeze(1)
            target_tensor = target_tensor.to(device)
            # input_tensor = source_data[iter-1]
            # input_tensor = input_tensor.to(device)
            # target_tensor = target_data[iter-1]
            # target_tensor = target_tensor.to(device)
            # #print(input_tensor)
            #return

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                # print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                #                             iter, iter / n_iters * 100, print_loss_avg))
                logger.info( f'{(timeSince(start, iter / n_iters))} ({iter} {int(iter / n_iters * 100)}%) {print_loss_avg}')
            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

        
        showPlot(plot_losses)
        evaluateRandomly(encoder1, attn_decoder1)
    return plot_losses

결과 도식화
----------------

matplotlib로 학습 중에 저장된 손실 값 ``plot_losses`` 의 배열을
사용하여 도식화합니다.




In [37]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

평가
==========

평가는 대부분 학습과 동일하지만 목표가 없으므로 각 단계마다 디코더의
예측을 되돌려 전달합니다.
단어를 예측할 때마다 그 단어를 출력 문자열에 추가합니다.
만약 EOS 토큰을 예측하면 거기에서 멈춥니다.
나중에 도식화를 위해서 디코더의 Attention 출력을 저장합니다.




In [38]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        #input_tensor = tensorFromSentence(input_lang, sentence)
        input_tensor = sentence.to(device)#datas['test_source.txt']
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            # if topi.item() == EOS_token:
            #     decoded_words.append('<EOS>')
            #     break
            # else:
            #     decoded_words.append(output_lang.index2word[topi.item()])
            if topi.item() == EOS_token:
                decoded_words.append(1)
                break
            else:
                decoded_words.append(topi.item())


            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

학습 세트에 있는 임의의 문장을 평가하고
입력, 목표 및 출력을 출력하여 주관적인 품질 판단을 내릴 수 있습니다:




In [39]:
import random
def evaluateRandomly(encoder, decoder, n=1):#10):
    for i in range(n):
        ran = random.randrange(1,len(datas['test_source.txt']))
        pair = [datas['test_source.txt']['source'][ran], datas['test_target.txt']['target'][ran]]
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, torch.Tensor(pair[0]).type(torch.long).unsqueeze(1))
        #output_sentence = ' '.join(output_words)
        print('<',output_words)
        #print('<', output_sentence)
        print('')

학습과 평가
=======================

이러한 모든 헬퍼 함수를 이용해서 (추가 작업처럼 보이지만 여러 실험을
더 쉽게 수행 할 수 있음) 실제로 네트워크를 초기화하고 학습을
시작할 수 있습니다.

입력 문장이 많이 필터링되었음을 기억하십시오. 이 작은 데이터 세트의
경우 256 크기의 은닉 노드(hidden node)와 단일 GRU 계층 같은 상대적으로 작은
네트워크를 사용할 수 있습니다. MacBook CPU에서 약 40분 후에
합리적인 결과를 얻을 것입니다.

.. Note::
   이 노트북을 실행하면 학습, 커널 중단, 평가를 할 수 있고 나중에
   이어서 학습을 할 수 있습니다. 인코더와 디코더가 초기화 된 행을
   주석 처리하고 ``trainIters`` 를 다시 실행하십시오.




In [40]:
#device = 'cpu'
EPOCHS = 10

In [41]:
hidden_size = 256
#encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
#attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
encoder1 = EncoderRNN(700, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, 700, dropout_p=0.1).to(device)



# 모델의 state_dict 출력
logger.info("ENCODER's state_dict:")
for param_tensor in encoder1.state_dict():
    logger.info(f"{param_tensor}\t {encoder1.state_dict()[param_tensor].size()}")

logger.info("DECODER's state_dict:")
for param_tensor in attn_decoder1.state_dict():
    logger.info(f"{param_tensor}\t {attn_decoder1.state_dict()[param_tensor].size()}")



plot_losses = trainIters(encoder1, attn_decoder1,  len(datas['train_source.txt']), print_every=100, logger=logger,epochs=EPOCHS)
logger.info("TRAIN FINISHED")
logger.info('='*50)

2021-06-10 13:29:23,940 - ENCODER's state_dict:
2021-06-10 13:29:23,942 - embedding.weight	 torch.Size([700, 256])
2021-06-10 13:29:23,944 - gru.weight_ih_l0	 torch.Size([768, 256])
2021-06-10 13:29:23,946 - gru.weight_hh_l0	 torch.Size([768, 256])
2021-06-10 13:29:23,948 - gru.bias_ih_l0	 torch.Size([768])
2021-06-10 13:29:23,949 - gru.bias_hh_l0	 torch.Size([768])
2021-06-10 13:29:23,950 - DECODER's state_dict:
2021-06-10 13:29:23,953 - embedding.weight	 torch.Size([700, 256])
2021-06-10 13:29:23,955 - attn.weight	 torch.Size([100, 512])
2021-06-10 13:29:23,957 - attn.bias	 torch.Size([100])
2021-06-10 13:29:23,959 - attn_combine.weight	 torch.Size([256, 512])
2021-06-10 13:29:23,961 - attn_combine.bias	 torch.Size([256])
2021-06-10 13:29:23,963 - gru.weight_ih_l0	 torch.Size([768, 256])
2021-06-10 13:29:23,964 - gru.weight_hh_l0	 torch.Size([768, 256])
2021-06-10 13:29:23,966 - gru.bias_ih_l0	 torch.Size([768])
2021-06-10 13:29:23,968 - gru.bias_hh_l0	 torch.Size([768])
2021-06-10 1

> [105, 140, 453, 35, 68, 78, 311, 68, 263, 105, 95, 140, 227, 271, 200, 311, 68, 33, 200, 95, 140, 311, 327, 1]
= [339, 551, 68, 43, 158, 68, 247, 85, 70, 158, 68, 79, 222, 1]
< [607, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 1]



2021-06-10 13:32:42,027 - 3m 18s (- 236m 20s) (100 1%) 5.471259771509255
2021-06-10 13:32:44,647 - 3m 20s (- 118m 3s) (200 2%) 3.415283876193912
2021-06-10 13:32:47,266 - 3m 23s (- 78m 36s) (300 4%) 3.525349118737818
2021-06-10 13:32:49,930 - 3m 25s (- 58m 52s) (400 5%) 3.521742285855019
2021-06-10 13:32:52,746 - 3m 28s (- 47m 2s) (500 6%) 3.352060354859046
2021-06-10 13:32:55,584 - 3m 31s (- 39m 8s) (600 8%) 3.478306037649738
2021-06-10 13:32:58,178 - 3m 34s (- 33m 27s) (700 9%) 3.541017283548758
2021-06-10 13:33:01,013 - 3m 37s (- 29m 12s) (800 11%) 3.5283671136529393
2021-06-10 13:33:03,584 - 3m 39s (- 25m 51s) (900 12%) 3.512636212258136
2021-06-10 13:33:06,186 - 3m 42s (- 23m 11s) (1000 13%) 3.41699686857367
2021-06-10 13:33:08,664 - 3m 44s (- 20m 58s) (1100 15%) 3.5228550274287715
2021-06-10 13:33:11,333 - 3m 47s (- 19m 8s) (1200 16%) 3.4130091097454254
2021-06-10 13:33:14,255 - 3m 50s (- 17m 35s) (1300 17%) 3.564300820971275
2021-06-10 13:33:16,920 - 3m 52s (- 16m 15s) (1400 19%

> [105, 68, 200, 33, 584, 416, 584, 402, 105, 23, 437, 68, 304, 584, 95, 68, 78, 311, 68, 263, 437, 227, 200, 156, 105, 584, 95, 68, 311, 437, 35, 140, 437, 95, 271, 200, 311, 327, 1]
= [607, 158, 68, 189, 192, 194, 440, 158, 85, 68, 241, 68, 43, 158, 68, 405, 344, 158, 332, 68, 276, 85, 299, 70, 158, 1]
< [607, 158, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 1]



2021-06-10 13:35:52,974 - 6m 29s (- 464m 12s) (100 1%) 5.447535121776284
2021-06-10 13:35:55,517 - 6m 31s (- 230m 21s) (200 2%) 3.355889547952502
2021-06-10 13:35:58,102 - 6m 34s (- 152m 23s) (300 4%) 3.298787291685213
2021-06-10 13:36:00,834 - 6m 36s (- 113m 26s) (400 5%) 3.474993887013086
2021-06-10 13:36:03,802 - 6m 39s (- 90m 5s) (500 6%) 3.1469792909680767
2021-06-10 13:36:06,653 - 6m 42s (- 74m 29s) (600 8%) 3.3024967914113947
2021-06-10 13:36:09,269 - 6m 45s (- 63m 18s) (700 9%) 3.2463728184016043
2021-06-10 13:36:12,084 - 6m 48s (- 54m 55s) (800 11%) 3.3291918467399597
2021-06-10 13:36:14,819 - 6m 50s (- 48m 23s) (900 12%) 3.407036843758249
2021-06-10 13:36:17,491 - 6m 53s (- 43m 8s) (1000 13%) 3.283404379973552
2021-06-10 13:36:19,962 - 6m 55s (- 38m 49s) (1100 15%) 3.244708413638267
2021-06-10 13:36:22,645 - 6m 58s (- 35m 14s) (1200 16%) 3.2604530391992057
2021-06-10 13:36:25,585 - 7m 1s (- 32m 12s) (1300 17%) 3.4504674904613597
2021-06-10 13:36:28,312 - 7m 4s (- 29m 36s) (14

...

> [78, 105, 35, 35, 437, 271, 1]
= [517, 415, 149, 1]
< [85, 149, 149, 149, 1]



2021-06-10 13:55:23,494 - 25m 59s (- 1861m 1s) (100 1%) 4.116090071144946
2021-06-10 13:55:26,166 - 26m 2s (- 919m 5s) (200 2%) 2.4104785887917886
2021-06-10 13:55:28,865 - 26m 4s (- 605m 5s) (300 4%) 2.4405114292545993
2021-06-10 13:55:31,606 - 26m 7s (- 448m 4s) (400 5%) 2.5158969150245163
2021-06-10 13:55:34,508 - 26m 10s (- 353m 53s) (500 6%) 2.2657398938721784
2021-06-10 13:55:37,382 - 26m 13s (- 291m 4s) (600 8%) 2.3485846250030518
2021-06-10 13:55:40,065 - 26m 16s (- 246m 10s) (700 9%) 2.3215833249941804
2021-06-10 13:55:43,005 - 26m 19s (- 212m 30s) (800 11%) 2.528535558272456
2021-06-10 13:55:45,681 - 26m 21s (- 186m 17s) (900 12%) 2.479211424784389
2021-06-10 13:55:48,393 - 26m 24s (- 165m 18s) (1000 13%) 2.314023787646386
2021-06-10 13:55:50,920 - 26m 26s (- 148m 6s) (1100 15%) 2.265524129701895
2021-06-10 13:55:53,689 - 26m 29s (- 133m 48s) (1200 16%) 2.2181097936898713
2021-06-10 13:55:56,665 - 26m 32s (- 121m 41s) (1300 17%) 2.326039009992956
2021-06-10 13:55:59,473 - 26m

> [157, 342, 78, 263, 1]
= [42, 1]
< [85, 1]



2021-06-10 13:58:40,198 - 29m 16s (- 2095m 45s) (100 1%) 3.762666727311568
2021-06-10 13:58:42,821 - 29m 18s (- 1034m 47s) (200 2%) 2.25013328051563
2021-06-10 13:58:45,546 - 29m 21s (- 681m 8s) (300 4%) 2.1988149271993573
2021-06-10 13:58:48,267 - 29m 24s (- 504m 17s) (400 5%) 2.3035184720811634
2021-06-10 13:58:51,149 - 29m 27s (- 398m 12s) (500 6%) 2.1183438489088653
2021-06-10 13:58:54,008 - 29m 30s (- 327m 27s) (600 8%) 2.031365488803601
2021-06-10 13:58:56,648 - 29m 32s (- 276m 52s) (700 9%) 2.103384647417431
2021-06-10 13:58:59,520 - 29m 35s (- 238m 57s) (800 11%) 2.3300465892660984
2021-06-10 13:59:02,142 - 29m 38s (- 209m 25s) (900 12%) 2.1462871467696827
2021-06-10 13:59:04,794 - 29m 40s (- 185m 47s) (1000 13%) 2.1587809077386835
2021-06-10 13:59:07,286 - 29m 43s (- 166m 26s) (1100 15%) 2.12869626633882
2021-06-10 13:59:09,977 - 29m 46s (- 150m 19s) (1200 16%) 2.1162763845792854
2021-06-10 13:59:12,914 - 29m 48s (- 136m 41s) (1300 17%) 2.1807125726274745
2021-06-10 13:59:15,7

> [52, 227, 200, 157, 402, 437, 1]
= [365, 430, 199, 1]
< [365, 430, 199, 1]



In [42]:
def corrects(encoder, decoder):#10):
    corrects = 0
    for i in range(len(datas['test_source.txt'])):
        ran = random.randrange(1,len(datas['test_source.txt']))
        pair = [datas['test_source.txt']['source'][ran], datas['test_target.txt']['target'][ran]]
        output_words, attentions = evaluate(encoder, decoder, torch.Tensor(pair[0]).type(torch.long).unsqueeze(1))
        if pair[1] == output_words:
            corrects+=1
    return corrects

In [43]:
torch.save(encoder1.state_dict(), './encoder_SGD.pth')
torch.save(attn_decoder1.state_dict(), './attn_decoder_SGD.pth')
logger.info(f"MODEL SAVED as : encoder_SGD.pth , attn_decoder_SGD.pth")

2021-06-10 14:10:15,792 - MODEL SAVED as : encoder_SGD.pth , attn_decoder_SGD.pth


In [44]:
from matplotlib import pyplot as plt
plt.figure()
fig, ax = plt.subplots()
# 주기적인 간격에 이 locator가 tick을 설정
loc = ticker.MultipleLocator(base=0.2)
ax.yaxis.set_major_locator(loc)
plt.plot(plot_losses)
plt.show()
plt.savefig('GRU_loss_SGD.png')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until
